In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu
import gw_movmtn as GW

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(GW)


In [ ]:
#####
exp_Val='ERA-5'

#user_C,user_X='juliob','juliob'

subd='hist'
subd='regridded'

exp_X,user_X = 'fmthist_MM_x21_2' , 'juliob'

if (user_X == 'juliob' ):
    path_X = '/glade/derecho/scratch/juliob/archive/'+exp_X+'/atm/'+ subd +'/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'tilmes' ):
    path_X = '/glade/derecho/scratch/tilmes/archive/'+exp_X+'/atm/hist/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'pel' ):
    path_X = '/glade/derecho/scratch/pel/archive/'+exp_X+'/atm/hist/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'juliob_run' ):
    path_X = '/glade/derecho/scratch/juliob/'+exp_X+'/run/'+exp_X+'.cam.h0.*.nc' 


In [ ]:
########################




X=xr.open_mfdataset( path_X , data_vars='different', coords='different' )

In [ ]:
X


In [ ]:
sqP = np.sqrt(X.UPWP_CLUBB**2+X.VPWP_CLUBB**2 )

X['MOMFLXMAG_CLUBB'] = sqP

X['TTEND_CLUBB'] = X['STEND_CLUBB'] * 86400./1002.

CLUBBparms = ['MOMFLXMAG_CLUBB','WP2_CLUBB','TTEND_CLUBB','THLP2_CLUBB']


In [ ]:
#########
season = 'Jul'
Ys={}
for parm in CLUBBparms:
    print(f" {parm} ")
    Ys[parm] = Av.Seasonal( ds=X, season=season , fld=parm)

In [ ]:


print(Ys['THLP2_CLUBB'].shape)

In [ ]:
######################
# Pretty latlon plot
######################

zlev=-7.0*np.log( X.lev.values/1_000. )

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
#MapProj = ccrs.Orthographic(central_longitude=180.,central_latitude=-90.)
DataProj = ccrs.PlateCarree()

"""
Add axes method 
ax = fig.add_axes([xmin , ymin , X , dy , ])
"""

fig = plt.figure(figsize=(20, 12))
levs=[29,26,20,10]
   
monAs=[ 'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec' ]    
mon=10-1
cmap='gist_ncar'
npo=0

#season=monAs[mon%12]

kl0,kl1=78,82
kl0,kl1=88,92

zA = f" ( {X.lev[kl0]:.0f}-{X.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

CoastColors=['white','white','white','black']
clevs = [np.linspace(0,0.2,21), np.linspace(0,1.0,21),np.linspace(-10,10,21),np.linspace(0,0.1,21),21]
cmaps = ['gist_ncar','gist_ncar','bwr','gist_ncar']

for CLUBBparm in CLUBBparms:
    npo=npo+1
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 

    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1],linewidth=2)


    #AAxy = np.average(X[CLUBBparm][mon,kl0:kl1,:,:],axis=0)
    AAxy = np.average(Ys[CLUBBparm][kl0:kl1,:,:],axis=0)

    co1=ax1.contourf(X.lon,X.lat, AAxy ,transform=DataProj,levels=clevs[npo-1],cmap=cmaps[npo-1] )

    cbar = plt.colorbar(co1, shrink=.6)
    #ax1.set_title( CLUBBparm + zA, fontsize=16)
    ax1.set_title( f"{season.upper()} {CLUBBparm} {zA}" , fontsize=16)




In [ ]:
########
nmos,nz,ny,nx = np.shape( X['MOMFLXMAG_CLUBB'] )
momflxmag_clubb = X['MOMFLXMAG_CLUBB'].values




In [ ]:

latS,latN=-60,-50
yS=np.argmin( np.abs(X.lat.values - latS ))
yN=np.argmin( np.abs(X.lat.values - latN ))
print( yS, yN)

print(X.lat[31])

momflxmag_clubb_a = np.average( np.average( momflxmag_clubb , axis=3 )[:,:,yS:yN+1], axis=2 )


In [ ]:

nyr=nmos//12
print(nyr)

momflxmag_clubb_a=momflxmag_clubb_a[0:nyr*12,:]
momflxmag_clubb_ar=momflxmag_clubb_a.reshape(nyr,12,93)     # after reshape, year is 0-th index
momflxmag_clubb_ann=np.average(momflxmag_clubb_ar,axis=0)  # So this average over axis=0 gives the 
                                                           # mean seasonal cycle


In [ ]:

#plt.contourf( np.arange(12)+1.,zlev, momflxmag_clubb_ann.T ,levels=21)
plt.contourf( monAs,zlev, momflxmag_clubb_ann.T ,levels=21)
plt.ylim(0,1)

In [ ]:
######################
# Pretty latlon plot
######################

zlev=-7.0*np.log( X.lev.values/1_000. )

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
#MapProj = ccrs.Orthographic(central_longitude=180.,central_latitude=-90.)
DataProj = ccrs.PlateCarree()

"""
Add axes method 
ax = fig.add_axes([xmin , ymin , X , dy , ])
"""

fig = plt.figure(figsize=(20, 12))
levs=[29,26,20,10]
   
monAs=[ 'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec' ]    
mon=12-1
cmap='gist_ncar'
npo=0

season=monAs[mon%12]

#kl0,kl1=78,82
zA = f" ( {X.lev[kl0]:.0f}-{X.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

CoastColors=['white','white','white','black']
clevs = [np.linspace(0,0.2,21), np.linspace(0,1.0,21),np.linspace(0,20,21),np.linspace(0,0.1,21),21]

for CLUBBparm in CLUBBparms:
    npo=npo+1
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 

    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1],linewidth=2)


    AAxy = np.average(X[CLUBBparm][mon,kl0:kl1,:,:],axis=0)

    co1=ax1.contourf(X.lon,X.lat, AAxy ,transform=DataProj,levels=clevs[npo-1],cmap=cmap)

    cbar = plt.colorbar(co1, shrink=.6)
    #ax1.set_title( CLUBBparm + zA, fontsize=16)
    ax1.set_title( f"{season.upper()} {CLUBBparm} {zA}" , fontsize=16)




In [ ]:
###

zMOM = np.average( X['MOMFLXMAG_CLUBB'] ,axis=3 )
#zMOM = np.average( X['U'] ,axis=3 )

In [ ]:
print(zlev[80:])

In [ ]:
print(X.lat[30].values)

In [ ]:

fig,ax=plt.subplots(1,2,figsize=(20,5))
#clev=21 #np.linspace(0,10,21)
clev=np.linspace(0,.08,21)
c0=ax[0].contourf( monAs,zlev,zMOM[:,:,30].T ,levels=clev )
fig.colorbar(c0,ax=ax[0],location='bottom')
ax[0].set_ylim(0,1)
c1=ax[1].contourf( monAs,zlev,zMOM[:,:,92-30].T , levels=clev)
fig.colorbar(c1,ax=ax[1],location='bottom')
ax[1].set_ylim(0,1)


In [ ]:

clev = np.linspace( 0., 1., num=21 )
plt.tricontourf(X.lon,X.lat,np.average( np.abs(X.UPWP_CLUBB_GW[1,90:,:] ),axis=0), levels=clev )
plt.colorbar()
#plt.contour(X.UBM_MOVMTN[5,:,:] )
#plt.xlim(19475,19500)
#plt.plot(X.TAU_MOVMTN[5,30,:] )

In [ ]:

plt.tricontourf(X.lon,X.lat,X.TAU_MOVMTN[5,80,:] )
plt.colorbar()
#plt.contour(X.UBM_MOVMTN[5,:,:] )
#plt.xlim(19475,19500)
#plt.plot(X.TAU_MOVMTN[5,30,:] )

In [ ]:


plt.plot(X.TAU_MOVMTN[5,1,:] )
plt.xlim(30600,30700)
#plt.xlim(2350,2400)

In [ ]:

# inputs need to shaped ncol,nz or ntime,nz. Hence the '.T' here
gwut = GW.tends_from_tau( tau=X.TAU_MOVMTN[5,:,:].values.T , 
                        ubm=X.UBM_MOVMTN[5,:,:].values.T , 
                        pint=X.PINT[5,:,:].values.T  )

In [ ]:
print(np.shape(gwut))

In [ ]:

zlev = -7.*np.log( X.lev/1_000.)
zilev = -7.*np.log( X.ilev/1_000.)

In [ ]:

icol=2360
icol=30600
icol=14447
print(X.CS1_MOVMTN[5,icol].values)
print(X.STEER_LEVEL_MOVMTN[5,icol].values)
plt.plot( X.UBM_MOVMTN[5,:,icol]  , zlev ,color='black')
plt.plot( X.U_MOVMTN_IN[5,:,icol] , zlev ,color='red' )
plt.plot( X.V_MOVMTN_IN[5,:,icol] , zlev ,color='green' )


In [ ]:

plt.plot( X.UTGW_MOVMTN[5,:,icol] , zlev ,color='red')
plt.plot( X.VTGW_MOVMTN[5,:,icol] , zlev ,color='green')
#plt.xlim(-.01,.01)


In [ ]:
plt.plot( X.GWUT_MOVMTN[5,:,icol] , zlev ,'o',color='green')
plt.plot( gwut[icol,:],zlev,'x',color='red')
#plt.ylim(0,50)

In [ ]:
plt.plot( X.GWUT_MOVMTN[5,:,:].values.flatten() ) # - gwut.T.flatten()  )
#plt.scatter( X.GWUT_MOVMTN[5,:,:].values.flatten()  , gwut.T.flatten()  )
plt.plot( gwut.T.flatten() )

In [ ]:
for k in np.arange(start=40,stop=30,step=-1):
    print( f"{k} {zlev[k] :.2f} {X.GWUT_MOVMTN[5,k,icol]}  ")

In [ ]:
plt.plot( X.TAU_MOVMTN[5,:,icol] , zilev ,color='green')


In [ ]:
plt.contourf( X.TAU_MOVMTN[5,:,:] ,levels=21 ) 
plt.xlim(14440,14460)


In [ ]:
#plt.plot(1.e5*X.TAU_MOVMTN[5,:,19481] )
plt.plot(X.SRC_LEVEL_MOVMTN[5,:] )
